# Exploring the Online Presence of Sub Restaurant Chains with Google Trends

Inspiration for this project came from [this article](https://towardsdatascience.com/telling-stories-with-google-trends-using-pytrends-in-python-a11e5b8a177).

Check out the DataViz on Tableau Public [here.](https://public.tableau.com/app/profile/ellis.hartley/viz/AGoogleTrendsAnalysisofSubChains/Dashboard1?publish=yes)

### Import Library & Connect to Google

In [46]:
# pip install pytrends
import pandas as pd
import numpy as np
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

### Keyword List

In [312]:
directory = "/Users/Sarah Pierce/Documents/Firehouse/"
filename = "Firehouse subs - Chain Sub Restaurants.csv"
kw_file = pd.read_csv(directory+filename)
kw_list = kw_file["Name"].values.tolist()

In [313]:
kw_file

,Name,Locations,Employees,Revenue (M)
0,Blimpie,156,1000.0,63.0
1,Schlotzsky's,330,NaN,NaN
2,Subway,37540,410000.0,16100.0
3,Quiznos,500,NaN,170.0
4,Firehouse Subs,1200,2000.0,695.0
5,Jersey Mikes,1973,NaN,1000.0
6,Which Wich,431,NaN,227.9
7,Jimmy Johns,2755,45000.0,3125.4
8,Potbelly Sandwich,474,6000.0,409.7
9,Jason's Deli,300,NaN,500.0


### Building Payload & Collecting Data of Keywords

In [49]:
def gtrends_overtime(full_list, key_ref, save_name="", directory="", category=0, time='all', loc=''):
    #iterate every item in list plus a keyword reference as the relative comparison
    df = pd.DataFrame()
    i = 0
    while i < 1:
        l = kw_list[i:(i+1)]
        l.append(key_ref)
        pytrends.build_payload(l, cat=category, timeframe=time, geo=loc, gprop='')
        df_time = pytrends.interest_over_time()
        df_time.reset_index(inplace=True)
        df = df_time
        df = df.drop(columns="isPartial")
        i += 1
    if i >= 1:
        while i < len(kw_list):
            l = kw_list[i:(i+1)]
            l.append(key_ref)
            pytrends.build_payload(l, cat=category, timeframe=time, geo=loc, gprop='')
            df_time = pytrends.interest_over_time()
            df_time.reset_index(inplace=True)
            df = df.merge(df_time, on='date', suffixes=("_"+str(int(i)),"_"+str(int(i+1))))
            df = df.drop(columns="isPartial")
            i += 1
    df = df[df.columns[~df.columns.str.startswith(key_ref)]]
    return pd.DataFrame(df)

In [50]:
df_overtime = gtrends_overtime(kw_list, "Panera Bread", "_US", directory,
                               category=276, time='all', loc='US')
df_overtime.tail()

,date,Blimpie,Schlotzsky's,Subway,Quiznos,Firehouse Subs,Jersey Mikes,Which Wich,Jimmy Johns,Potbelly Sandwich,Jason's Deli
213,2021-10-01,1,0,74,4,15,21,1,47,1,1
214,2021-11-01,1,0,76,4,14,21,2,46,2,1
215,2021-12-01,1,0,82,4,14,22,1,51,1,1
216,2022-01-01,1,0,81,4,16,21,1,49,1,1
217,2022-02-01,1,0,75,4,17,20,1,51,1,2


In [51]:
def gtrends_region(kw_list, key_ref, save_name="", directory="",
                   category=0, time='all', loc="", res='COUNTRY'):
    #iterate every 4 item in list plus a keyword reference as the relative comparison
    df = pd.DataFrame()
    i = 0
    while i < 1:
        l = kw_list[i:(i+1)]
        l.append(key_ref)
        pytrends.build_payload(l, cat=category, timeframe=time, geo=loc, gprop='')
        # resolution can be set to 'REGION' when loc is set to specific country, e.g. 'US'
        df_region = pytrends.interest_by_region(resolution=res, inc_low_vol=True, inc_geo_code=False)
        df_region.reset_index(inplace=True)
        df = df_region
        i += 1
    if i >= 1:
        while i < len(kw_list):
            l = kw_list[i:(i+1)]
            l.append(key_ref)
            pytrends.build_payload(l, cat=category, timeframe=time, geo=loc, gprop='')
            df_region = pytrends.interest_by_region(resolution=res, inc_low_vol=True, inc_geo_code=False)
            df_region.reset_index(inplace=True)
            df = df.merge(df_region, on='geoName', suffixes=("_"+str(int(i)),"_"+str(int(i+1))))
            i += 1
    df = df[df.columns[~df.columns.str.startswith(key_ref)]]
    return pd.DataFrame(df)

In [52]:
df_region = gtrends_region(kw_list, "Panera Bread", "_US_", directory,
                           category=276, time='all', loc='US', res='REGION')
df_region.head()

,geoName,Blimpie,Schlotzsky's,Subway,Quiznos,Firehouse Subs,Jersey Mikes,Which Wich,Jimmy Johns,Potbelly Sandwich,Jason's Deli
0,Alabama,0,0,64,6,29,6,4,40,0,4
1,Alaska,0,4,97,73,66,0,0,54,0,0
2,Arizona,4,2,74,19,31,22,4,68,4,5
3,Arkansas,0,2,77,21,37,6,3,52,2,6
4,California,1,0,72,19,9,22,4,28,0,0


### Integration, Normalization & Cleaning Data

In [66]:
def tidy(df, kw_file, col=''):
    df=pd.melt(df,id_vars=[col],var_name='Name', value_name='RelativePopularity')
    df = df.merge(kw_file, on="Name")
    return df

In [67]:
overtime_US = tidy(df_overtime, kw_file, col='date')
# overtime_US.to_csv(directory+"gtrends_overtime_US.csv", index=False)

In [68]:
overtime_US

,date,Name,RelativePopularity,Locations,Employees,Revenue (M)
0,2004-01-01,Blimpie,2,156,1000.0,63.0
1,2004-02-01,Blimpie,2,156,1000.0,63.0
2,2004-03-01,Blimpie,0,156,1000.0,63.0
3,2004-04-01,Blimpie,2,156,1000.0,63.0
4,2004-05-01,Blimpie,2,156,1000.0,63.0
...,...,...,...,...,...,...
2175,2021-10-01,Jason's Deli,1,300,NaN,500.0
2176,2021-11-01,Jason's Deli,1,300,NaN,500.0
2177,2021-12-01,Jason's Deli,1,300,NaN,500.0
2178,2022-01-01,Jason's Deli,1,300,NaN,500.0


In [69]:
geo_US = tidy(df_region, kw_file, col='geoName')
geo_US

,geoName,Name,RelativePopularity,Locations,Employees,Revenue (M)
0,Alabama,Blimpie,0,156,1000.0,63.0
1,Alaska,Blimpie,0,156,1000.0,63.0
2,Arizona,Blimpie,4,156,1000.0,63.0
3,Arkansas,Blimpie,0,156,1000.0,63.0
4,California,Blimpie,1,156,1000.0,63.0
...,...,...,...,...,...,...
505,Virginia,Jason's Deli,3,300,NaN,500.0
506,Washington,Jason's Deli,0,300,NaN,500.0
507,West Virginia,Jason's Deli,0,300,NaN,500.0
508,Wisconsin,Jason's Deli,1,300,NaN,500.0


In [70]:
geo = geo_US.fillna(0)
geoSum = pd.DataFrame(geo.groupby(['geoName']).sum()).reset_index()
geoSum = geoSum[["geoName","RelativePopularity"]]
geoSum.columns = ['geoName', 'geoSum']
geoSum

# def calculate_percent(df_geo)

,geoName,geoSum
0,Alabama,153
1,Alaska,294
2,Arizona,233
3,Arkansas,206
4,California,155
5,Colorado,187
6,Connecticut,84
7,Delaware,87
8,District of Columbia,137
9,Florida,145


In [71]:
geo = pd.merge(geo, geoSum, on=['geoName'])
geo['PercentageBreakdown'] = round((geo['RelativePopularity']/geo['geoSum'])*100,2)
geo.iloc[:,4:] = geo.iloc[:,4:].astype(int)
# geo.to_csv(directory + "gtrends_US.csv", index = False)

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
geo.to_sql(name="geo", con=engine, if_exists='replace')

geo

,geoName,Name,RelativePopularity,Locations,Employees,Revenue (M),geoSum,PercentageBreakdown
0,Alabama,Blimpie,0,156,1000,63,153,0
1,Alabama,Schlotzsky's,0,330,0,0,153,0
2,Alabama,Subway,64,37540,410000,16100,153,41
3,Alabama,Quiznos,6,500,0,170,153,3
4,Alabama,Firehouse Subs,29,1200,2000,695,153,18
...,...,...,...,...,...,...,...,...
505,Wyoming,Jersey Mikes,18,1973,0,1000,263,6
506,Wyoming,Which Wich,0,431,0,227,263,0
507,Wyoming,Jimmy Johns,89,2755,45000,3125,263,33
508,Wyoming,Potbelly Sandwich,0,474,6000,409,263,0


In [306]:
#create a rank column and save
ranked_states = pd.read_sql('''
SELECT geoName, Name, Locations, RelativePopularity, PercentageBreakdown, 
RANK() OVER (PARTITION by geoName ORDER BY PercentageBreakdown DESC) AS Rank
FROM geo 
ORDER BY geoName, PercentageBreakdown DESC
''',con=engine)
ranked_states.to_sql(name="ranked_states", con=engine, if_exists='replace')
ranked_regions_US.to_csv(directory + "gtrends_states_ranked.csv", index = False)

In [146]:
ranked_states[(ranked_states["Name"]=="Schlotzsky's") & (ranked_states["RelativePopularity"] > 0)]

,geoName,Name,RelativePopularity,Locations,PercentageBreakdown,Rank
14,Alaska,Schlotzsky's,4,330,1,5
29,Arizona,Schlotzsky's,2,330,0,10
38,Arkansas,Schlotzsky's,2,330,0,8
168,Kansas,Schlotzsky's,2,330,0,8
188,Louisiana,Schlotzsky's,1,330,0,8
248,Mississippi,Schlotzsky's,1,330,0,8
259,Missouri,Schlotzsky's,1,330,0,9
278,Nebraska,Schlotzsky's,1,330,0,9
318,New Mexico,Schlotzsky's,1,330,0,8
347,North Dakota,Schlotzsky's,2,330,0,7


In [141]:
pd.read_sql('''
SELECT Name, SUM(RelativePopularity) SumRelativePop, Locations, SUM(PercentageBreakdown) SumPct, AVG(Rank) AvgRank
FROM ranked_states 
GROUP BY Name
ORDER BY AvgRank
''',con=engine).to_sql(name="avg", con=engine, if_exists='replace')

In [161]:
rank_overall = pd.read_sql('''
SELECT Name, Locations, SumRelativePop RelativePopularity, 
ROUND(SumPct*100.0 / SUM(SumPct) OVER(), 1) PctDominance, AvgRank,
RANK() OVER (ORDER BY AvgRank) OverallRank
FROM avg 
''',con=engine)
rank_overall.to_csv(directory + "gtrends_rank_overall.csv", index = False)
rank_overall

,Name,Locations,RelativePopularity,PctDominance,AvgRank,OverallRank
0,Subway,37540,3546,42.4,1.235294,1
1,Jimmy Johns,2755,2649,27.3,1.921569,2
2,Firehouse Subs,1200,1029,10.3,3.666667,3
3,Quiznos,500,937,9.2,3.862745,4
4,Jersey Mikes,1973,659,7.2,4.254902,5
5,Which Wich,431,161,1.3,6.352941,6
6,Jason's Deli,300,147,1.2,6.392157,7
7,Blimpie,156,112,0.8,6.941176,8
8,Potbelly Sandwich,474,64,0.3,7.039216,9
9,Schlotzsky's,330,30,0.1,7.392157,10


In [200]:
overtime_US

,date,Name,RelativePopularity,Locations,Employees,Revenue (M)
0,2004-01-01,Blimpie,2,156,1000.0,63.0
1,2004-02-01,Blimpie,2,156,1000.0,63.0
2,2004-03-01,Blimpie,0,156,1000.0,63.0
3,2004-04-01,Blimpie,2,156,1000.0,63.0
4,2004-05-01,Blimpie,2,156,1000.0,63.0
...,...,...,...,...,...,...
2175,2021-10-01,Jason's Deli,1,300,NaN,500.0
2176,2021-11-01,Jason's Deli,1,300,NaN,500.0
2177,2021-12-01,Jason's Deli,1,300,NaN,500.0
2178,2022-01-01,Jason's Deli,1,300,NaN,500.0


In [204]:
overtime_US.to_sql(name="time", con=engine, if_exists='replace')

year = pd.read_sql('''
SELECT cast(date as datetime) year, Name, AVG(RelativePopularity) RelativePopularity
FROM time
group by Name, year
order by Name, year
''', con=engine)
year.to_sql(name="year", con=engine, if_exists='replace')
year

,year,Name,RelativePopularity
0,2004,Blimpie,1.333333
1,2005,Blimpie,1.750000
2,2006,Blimpie,1.250000
3,2007,Blimpie,0.916667
4,2008,Blimpie,1.083333
...,...,...,...
185,2018,Which Wich,3.250000
186,2019,Which Wich,3.250000
187,2020,Which Wich,2.250000
188,2021,Which Wich,1.916667


In [308]:
time_yoy = pd.read_sql('''
SELECT year Year, Name, Round(RelativePopularity, 1) RelativePopularity, 
Round((RelativePopularity - LAG(RelativePopularity) 
OVER (PARTITION BY Name ORDER BY Name))
/LAG(RelativePopularity) 
OVER (PARTITION BY Name ORDER BY Name),2)*100 YoY_Pct_Growth
FROM year
order by Name
''', con=engine)
time_yoy = time_yoy.fillna(0)
time_yoy.to_csv(directory + "gtrends_time_yoy.csv", index = False)
time_yoy.head(22)

,Year,Name,RelativePopularity,YoY_Pct_Growth
0,2004,Blimpie,1.3,0.0
1,2005,Blimpie,1.8,31.0
2,2006,Blimpie,1.3,-29.0
3,2007,Blimpie,0.9,-27.0
4,2008,Blimpie,1.1,18.0
5,2009,Blimpie,1.2,8.0
6,2010,Blimpie,0.9,-21.0
7,2011,Blimpie,1.3,45.0
8,2012,Blimpie,1.2,-12.0
9,2013,Blimpie,1.2,0.0


### Check out the DataViz in Tableau public: 
https://public.tableau.com/app/profile/ellis.hartley/viz/AGoogleTrendsAnalysisofSubChains/Dashboard1https://public.tableau.com/app/profile/ellis.hartley/viz/AGoogleTrendsAnalysisofSubChains/Dashboard1